<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 10. Design Patterns with First-Class Functions

Strategy Pattern: Define a family of algorithms, encapsulate each one, and make them inerchangeable. Strategy lets the algorithm vary independently from clients that use it.

In [ ]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional

class Customer(NamedTuple):
  name: str
  fidelity: str

class LineItem(NamedTuple):
  product: str
  quantity: int
  price: Decimal

  def total(self) -> Decimal:
    return self.price * self.quantity

class Order(NamedTuple): # context
  customer: Customer
  cart: Sequence[LineItem]
  promotion: Optional['Promotion'] = None

  def total(self) -> Decimal:
    totals = (item.total() for item in self.cart)
    return sum(totals, start=Decimal(0))

  def due(self) -> Decimal:
    if self.promotion is None:
      discount = Decimal(0)
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

class Promotion(ABC): # Strategy: an abstract base class
  @abstractmethod
  def discount(self, order: Order) -> Decimal:
    """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion):

  def discount(self, order: Order) -> Decimal:
    rate = Decimal('0.05')
    if order.customer.fidelity >= 1000:
      return order.total() * rate
    return Decimal(0)

class BulkItemPromo(Promotion):

  def discount(self, order: Order) -> Decimal:
    discount = Decimal(0)
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * Decimal('0.1')
    return discount

class LargeOrderPromo(Promotion):
  """ 7% discount for orders with 10 or more distinct items"""
  def discount(self, order: Order) -> Decimal:
    distinct_items = { item.product for item in order.cart }
    if len(distinct_items) >= 10:
      return order.total() * Decimal('0.07')
    return Decimal(0)

In [ ]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [ ]:
cart = (LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)),)

In [ ]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [ ]:
banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('apple', 10, Decimal('1.5')))

In [ ]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [ ]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

In [ ]:
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-Oriented Strategy

Observation: each concrete strategy is a class with a single method, `discount`. Furthermore, strategy instances have no state.

In [2]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import NamedTuple, Optional, Callable

In [3]:
class Customer(NamedTuple):
  name: str
  fidelity: str

class LineItem(NamedTuple):
  product: str
  quantity: int
  price: Decimal

  def total(self) -> Decimal:
    return self.price * self.quantity

@dataclass(frozen=True)
class Order: # the context
  customer: Customer
  cart: Sequence[LineItem]
  # This type hint says: promotion may be None or
  # it may be a callable that takes an `Order` arg and returns a Decimal
  promotion: Optional[Callable[['Order'], Decimal]] = None

  def total(self) -> Decimal:
    totals = (item.total() for item in self.cart)
    return sum(totals, start=Decimal(0))

  def due(self) -> Decimal:
    if self.promotion is None:
      discount = Decimal(0)
    else:
      # Call the `self.promotion` callable,
      # passing self as an argument
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

def fidelity_promo(order: Order) -> Decimal:
  if order.customer.fidelity >= 1000:
    return order.total() * Decimal('0.05')
  return Decimal(0)

def bulk_item_promo(order: Order) -> Decimal:
  discount = Decimal(0)
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * Decimal('0.1')
  return discount

def large_order_promo(order: Order) -> Decimal:
  distinct_items = { item.product for item in order.cart }
  if len(distinct_items) >= 10:
    return order.total() * Decimal('0.07')
  return Decimal(0)

In [5]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [11]:
cart = (LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)),)
banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('apple', 10, Decimal('1.5')))
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

In [ ]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [ ]:
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

## Choosing best strategy: simple approach

In [4]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order: Order) -> Decimal:
  return max(promo(order) for promo in promos)

In [ ]:
Order(joe)

# Finding Strategies in a Module

Modules are also first-class objects

`globals()`
 - Returns a dict representing the current global symbol table. This is always the dictionary of the current module (inside a fcn, or method, this is the module where it is defined)

In [8]:
promos = [promo for name, promo in globals().items()
                if name.endswith('_promo') and
                   name != 'best_promo'
]

In [9]:
def best_promo(order: Order) -> Decimal:
  return max(promo(order) for promo in promos)

In [14]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

Another way of collecting the available promotions would be to create a module and put all the strategy fcns there, except for `best_promo`.

In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
from decimal import Decimal
import inspect

from strategy import Order
import promotions

promos = [func for _, func in inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order: Order) -> Decimal:
  return max(promo(order) for promo in promos)

## Decorator-Enhanced Strategy Pattern

In [17]:
Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = [] # promos list is a module global

def promotion(promo: Promotion) -> Promotion:
  promos.append(promo)
  return promo

def best_promo(order: Order) -> Decimal:
  return max(promo(order) for promo in promos)

@promotion
def fidelity(order: Order) -> Decimal:
  if order.customer.fidelity >= 1000:
    return order.total() * Decimal('0.05')
  return Decimal(0)

@promotion
def bulk_item(order: Order) -> Decimal:
  discount = Decimal(0)
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * Decimal('0.1')
  return discount

@promotion
def large_order(order: Order) -> Decimal:
  distinct_items = { item.product for item in order.cart }
  if len(distinct_items) >= 10:
    return order.total() * Decimal('0.07')
  return Decimal(0)

In [18]:
print(Order(joe, banana_cart, best_promo))
print(Order(joe, long_cart, best_promo))
print(Order(ann, cart, best_promo))

<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 39.90>


Advantages of this solution over other solutions:
 * The promotion strategy fcns don't have to use special names
 * `@promotion` decorator highlights the purpose of the decorated fcn, and also makes it easy to temporarily disable a promotion
 * Promotional discount strategies may be defined in other modules, anywhere in the system, as long as the `@promotion` decoration is applied to them.

# Command Pattern

another design pattern that can be simplified by the use of fcn passed as args.

Goal of Command: decouple an object that invokes an operation (the invoker) from the provider object that implements it (the receiver)

In [ ]:
class MacroCommand:
  def __init__(self, commands):
    self.commands = list(commands)

  def __call__(self):
    for command in self.commands:
      command()

More advanced uses of the Command pattern (to support undo, for example) may require more than a simple callback fcn.